In [58]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['test', 'train', 'train.csv', 'sample_submission.csv']


In [59]:
import os
import numpy as np
from fastai import *
from fastai.vision import *

In [60]:
train_path = Path('../input/train')
test_path = Path('../input/test')
print(train_path, test_path)


../input/train ../input/test


In [61]:
import pandas as pd

train_df = pd.read_csv('../input/train.csv')
train_df.head(10)

,id,has_cactus
0,0004be2cfeaba1c0361d39e2b000257b.jpg,1
1,000c8a36845c0208e833c79c1bffedd1.jpg,1
2,000d1e9a533f62e55c289303b072733d.jpg,1
3,0011485b40695e9138e92d0b3fb55128.jpg,1
4,0014d7a11e90b62848904c1418fc8cf2.jpg,1
5,0017c3c18ddd57a2ea6f9848c79d83d2.jpg,1
6,002134abf28af54575c18741b89dd2a4.jpg,0
7,0024320f43bdd490562246435af4f90b.jpg,0
8,002930423b9840e67e5a54afd4768a1e.jpg,1
9,00351838ebf6dff6e53056e00a1e307c.jpg,1


In [62]:
test_df = pd.read_csv('../input/sample_submission.csv')
# test_df.head(10)
# print(len(test_df), len(train_df))

In [63]:
test_data = ImageList.from_df(test_df, path = test_path, folder = 'test')

train_data = (ImageList.from_df(train_df, path = train_path, folder = 'train')
            .split_by_rand_pct(0.1)
            .label_from_df()
            .add_test(test_data)
            .transform(get_transforms(), size = 32)
            .databunch(path = '.', bs = 64)
            .normalize(imagenet_stats))

In [64]:
learner = cnn_learner(train_data, models.densenet161, metrics = [error_rate, accuracy])

In [65]:
learner.fit_one_cycle(8)

epoch,train_loss,valid_loss,error_rate,accuracy,time
0,0.131820,0.070420,0.015429,0.984571,00:39
1,0.064914,0.119344,0.013143,0.986857,00:38
2,0.051661,0.122634,0.020000,0.980000,00:40
3,0.037830,0.014318,0.004000,0.996000,00:38
4,0.024627,0.017961,0.007429,0.992571,00:39
5,0.019074,0.012191,0.005143,0.994857,00:39
6,0.016776,0.007820,0.003429,0.996571,00:39
7,0.015802,0.012433,0.006286,0.993714,00:38


In [66]:
learner.unfreeze()
learner.fit_one_cycle(4)

epoch,train_loss,valid_loss,error_rate,accuracy,time
0,0.036848,0.039203,0.013714,0.986286,00:46
1,0.032397,0.010588,0.003429,0.996571,00:45
2,0.020449,0.013768,0.004571,0.995429,00:45
3,0.007261,0.008967,0.002286,0.997714,00:46


In [67]:
preds, _ = learner.get_preds(ds_type = DatasetType.Test)
test_df.has_cactus = preds.numpy()[:, 0]

In [68]:
test_df.to_csv('submission.csv', index = False)